# NI Acquisition, generation and visualization

In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ
import numpy as np

In [3]:
# Create input and output tasks:

# NI 9234
task_in = LDAQ.national_instruments.NITask("TestInputTask_2", sample_rate=25600)
task_in.add_channel(channel_name="V0", device_ind=2, channel_ind=0, sensitivity=None, sensitivity_units=None, units="V", scale=1., min_val=-4, max_val=4.0)
task_in.add_channel(channel_name="V1", device_ind=2, channel_ind=1, sensitivity=None, sensitivity_units=None, units="V", scale=1., min_val=-4, max_val=4.0)

# NI 9260
task_out = LDAQ.national_instruments.NITaskOutput("TestOutputTask_2", sample_rate=25600)
task_out.add_channel(channel_name="V0_out", device_ind=1, channel_ind=0, min_val=-4, max_val=4)
task_out.add_channel(channel_name="V2_out", device_ind=1, channel_ind=1, min_val=-4, max_val=4)

In [4]:
task_in

Task name: TestInputTask_2
Connected devices:
	(0) - cDAQ4
	(1) - cDAQ4Mod1
	(2) - cDAQ4Mod2
	(3) - VirtualDevice
	(4) - VirtualDevice2
Channels: ['V0', 'V1']

In [5]:
#create signal:
fs = 25600 # output sample rate
mp = 10

t = np.arange(fs * mp) / fs  
signal1 = np.sin(2*np.pi*800*t) 
signal2 = np.sin(2*np.pi*450*t) 

excitation_signal = np.array([signal1, signal2]).T # exitation signal that generation class will use

In [6]:
# create instances:
gen = LDAQ.national_instruments.NIGeneration(task_out, excitation_signal)
gen.add_delay(2.0) # generation will start 2 seconds after run() is called on the Core instance

acq = LDAQ.national_instruments.NIAcquisition(task_in, acquisition_name="NI")
vis = LDAQ.Visualization(refresh_rate=100)

# configure live visualization:
vis.add_lines((0,0), "NI", [0]) # Time signal V0
vis.add_lines((1,0), "NI", [1]) # Time signal V1
vis.add_lines((2,0), "NI", [0], function="fft", refresh_rate=3000) # Fourier transform of V0
vis.add_lines((3,0), "NI", [1], function="fft", refresh_rate=3000) # Fourier transform of V1

vis.config_subplot((0, 0), t_span=0.05, ylim=(-10, 10))
vis.config_subplot((1, 0), t_span=0.05, ylim=(-10, 10))
vis.config_subplot((2, 0), t_span=5.0, ylim=(0, 1.2), xlim=(200, 1000))
vis.config_subplot((3, 0), t_span=5.0, ylim=(0, 1.2), xlim=(200, 1000))

# create core object and add acquisition sources:
ldaq = LDAQ.Core(acquisitions=[acq], generations=[gen], visualization=vis)

# run acquisition:
ldaq.run(10.)

In [7]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement["NI"]

{'time': array([56.40003906, 56.40007813, 56.40011719, ..., 59.99988281,
        59.99992187, 59.99996094]),
 'channel_names': ['V0', 'V1'],
 'data': array([[-5.03444934e-01, -9.94202753e-01],
        [-6.62328816e-01, -9.77152079e-01],
        [-7.95702724e-01, -9.48246068e-01],
        ...,
        [ 3.82937440e-01, -9.13326268e-01],
        [ 1.95559480e-01, -9.52440940e-01],
        [ 6.84911256e-04, -9.80014218e-01]]),
 'sample_rate': 25600}